# Environment:

In [1]:
import numpy as np
import pandas as pd
import os
import json
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Vector Database:

In [3]:
with open('e5_explanation_vectors.json', 'r') as f:
    vectors = json.load(f)

BERT Tokenizer:

In [4]:
!pip install sentence_transformers

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("intfloat/e5-base")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [6]:
def get_vector(text):
    vector = model.encode(text, convert_to_tensor=True)
    return vector.cpu().numpy()

Explanations Data:

In [7]:
df = pd.read_csv('cleaned_explanations.csv')

LLM for text reply:

In [8]:
!pip install huggingface-hub
from huggingface_hub import notebook_login
notebook_login()

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
llama_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

# Vectorizing Query:

In [10]:
query = input("Enter your question:")

Enter your question:What does Thirukural say about honesty


In [12]:
query_vector = get_vector(query)

# Retrieval:

## Similarity with vectors:

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = {idx: cosine_similarity([query_vector], [vec])[0][0] for idx, vec in vectors.items()}

In [14]:
top_k = sorted(similarities, key=similarities.get, reverse=True)[:5]
top_k

['297', '299', '951', '1110', '178']

## Relevant Kurals:

In [15]:
top_kurals = [df['Verse'].iloc[int(idx)] for idx in top_k]
top_explanations = [df['Explanation'].iloc[int(idx)] for idx in top_k]

for i in range(len(top_k)):
    print("Kural",top_k[i],":", top_kurals[i])
    print("Explanation: ", top_explanations[i])
    print("")

Kural 297 : புறள்தூய்மை நீரான் அமையும் அகந்தூய்மை   வாய்மையால் காணப் படும். 
Explanation:  Purity of body is produced by water and purity of mind by truthfulness. 

Kural 299 : யாமெய்யாக் கண்டவற்றுள் இல்லை எனைத்தொன்றும்   வாய்மையின் நல்ல பிற. 
Explanation:  Amidst all that we have seen (described) as real (excellence), there is nothing so good as truthfulness. 

Kural 951 : ஒழுக்கமும் வாய்மையும் நாணும்இம் மூன்றும்   இழுக்கார் குடிப்பிறந் தார். 
Explanation:  The high-born will never deviate from these three; good manners, truthfulness and modesty. 

Kural 1110 : நன்னீரை வாழி அனிச்சமே நின்னினும்   மென்னீரள் யாம்வீழ் பவள். 
Explanation:  May you flourish, O Anicham! you have a delicate nature. But my beloved is more delicate than you. 

Kural 178 : அறனறிந்து வெஃகா அறிவுடையார்ச் சேரும்   திறன்அறிந் தாங்கே திரு. 
Explanation:  Lakshmi, knowing the manner (in which she may approach) will immediately come to those wise men who, knowing that it is virtue, covet not the property of others. 



# Generation:

## Prompt with Relevant Kurals:

In [48]:
def create_prompt(query, retrieved_kurals):
    prompt = f"QUESTION: {query}\n\n"
    prompt += "TEXT:\n"
    for i in range(5):
        prompt += f"{retrieved_kurals[i]}\n"
    prompt += "\nBased on the text, answer the question.\n"
    return prompt

## Generating Output with Llama:

In [49]:
def generate_response(query, retrieved_kurals):

    prompt = create_prompt(query, retrieved_kurals)

    inputs = llama_tokenizer(prompt, return_tensors="pt").to("cuda")
    output = llama_model.generate(inputs['input_ids'], max_length=350, do_sample=True, top_k=50, top_p=0.95)
    response = llama_tokenizer.decode(output[0], skip_special_tokens=True)

    #Return only the generated text
    generated_text = response[len(prompt):].strip()
    return generated_text

In [50]:
output_text = generate_response(query, top_explanations)
output_text

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'ANSWER: \nThirukural says that honesty is the most valuable virtue among all. It is the most valuable because it produces purity of mind. It is also one of the three virtues that the high-born will never deviate from, alongside good manners and modesty. Furthermore, Lakshmi, the goddess of prosperity, rewards those who possess honesty by coming to them immediately. This suggests that honesty is a key to attracting prosperity and good fortune. Overall, the text portrays honesty as a highly valued virtue that produces positive outcomes in both personal and spiritual life.'

## Final Output:

In [51]:
print("QUESTION:")
print(query)
print("\nRELEVANT KURALS:")
for i in range(5):
    print(top_kurals[i])
    print(top_explanations[i])
print("\n")
print(output_text)

QUESTION:
What does Thirukural say about honesty

RELEVANT KURALS:
புறள்தூய்மை நீரான் அமையும் அகந்தூய்மை   வாய்மையால் காணப் படும். 
Purity of body is produced by water and purity of mind by truthfulness. 
யாமெய்யாக் கண்டவற்றுள் இல்லை எனைத்தொன்றும்   வாய்மையின் நல்ல பிற. 
Amidst all that we have seen (described) as real (excellence), there is nothing so good as truthfulness. 
ஒழுக்கமும் வாய்மையும் நாணும்இம் மூன்றும்   இழுக்கார் குடிப்பிறந் தார். 
The high-born will never deviate from these three; good manners, truthfulness and modesty. 
நன்னீரை வாழி அனிச்சமே நின்னினும்   மென்னீரள் யாம்வீழ் பவள். 
May you flourish, O Anicham! you have a delicate nature. But my beloved is more delicate than you. 
அறனறிந்து வெஃகா அறிவுடையார்ச் சேரும்   திறன்அறிந் தாங்கே திரு. 
Lakshmi, knowing the manner (in which she may approach) will immediately come to those wise men who, knowing that it is virtue, covet not the property of others. 


ANSWER: 
Thirukural says that honesty is the most valuable virtue am